<a href="https://colab.research.google.com/github/asma-mo/Estate-Elite-Insights/blob/asma_branch/KPIs/python_code/health_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# calculating health_score measure

## Load and prepare our data

In [3]:
import pandas as pd
poverty = pd.read_csv("/content/poverty_rate.csv")
enviromental_health = pd.read_csv("/content/enviromental_health.csv")

In [33]:
poverty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34790 entries, 0 to 34793
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   poverty_rate                      34790 non-null  float64
 1   city_name                         34790 non-null  object 
 2   city_name_normalized              34790 non-null  object 
 3   municipality_code                 34790 non-null  object 
 4   latitude                          34790 non-null  float64
 5   longitude                         34790 non-null  float64
 6   normalized_poverty_rate           34790 non-null  float64
 7   inverted_normalized_poverty_rate  34790 non-null  float64
dtypes: float64(5), object(3)
memory usage: 2.4+ MB


In [34]:
enviromental_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   municipality_code          640 non-null    object 
 1   city_name                  640 non-null    object 
 2   operation_cost             640 non-null    float64
 3   normalized_operation_cost  640 non-null    float64
dtypes: float64(2), object(2)
memory usage: 20.1+ KB


In [4]:
poverty.head()

,poverty_rate,city_name,city_name_normalized,municipality_code,latitude,longitude
0,9.2,Ayse,AYSE,74024,46.086992,6.446016
1,9.2,Bonneville,BONNEVILLE,74042,46.073900,6.408685
2,9.2,Brizon,BRIZON,74049,46.035261,6.434152
3,9.2,Contamine-sur-Arve,CONTAMINE SUR ARVE,74087,46.133459,6.338695
4,9.2,Marignier,MARIGNIER,74164,46.095029,6.494469


In [5]:
enviromental_health.head()

,municipality_code,city_name,operation_cost
0,01004,Ambérieu-en-Bugey,251529.61
1,02142,Castres,5878227.00
2,02168,Château-Thierry,530884.00
3,02372,Hartennes-et-Taux,970368.95
4,02422,Lesquielles-Saint-Germain,5885000.00


### drop null rows from poverty

In [6]:
poverty.isnull().sum()

poverty_rate            0
city_name               4
city_name_normalized    4
municipality_code       4
latitude                4
longitude               4
dtype: int64

In [9]:
poverty.dropna(inplace=True)

In [10]:
poverty.isnull().sum()

poverty_rate            0
city_name               0
city_name_normalized    0
municipality_code       0
latitude                0
longitude               0
dtype: int64

In [11]:
enviromental_health.isnull().sum()

municipality_code    0
city_name            0
operation_cost       0
dtype: int64

## Perform normalization

In [21]:
from sklearn.preprocessing import StandardScaler

# MinMaxScaler from scikit-learn is scaling the data to a default range of 0 to 1.

scaler = StandardScaler()

In [26]:
# .values transform tha column datafram we have into an array to apply reshapes(-1,1)
poverty["normalized_poverty_rate"] = scaler.fit_transform(poverty["poverty_rate"].values.reshape(-1, 1))
poverty["normalized_poverty_rate"]

0       -1.019323
1       -1.019323
2       -1.019323
3       -1.019323
4       -1.019323
           ...   
34789    4.686590
34790    4.686590
34791    4.570616
34792    4.570616
34793    4.570616
Name: normalized_poverty_rate, Length: 34790, dtype: float64

In [25]:
enviromental_health["normalized_operation_cost"]= scaler.fit_transform(enviromental_health["operation_cost"].values.reshape(-1, 1))
enviromental_health["normalized_operation_cost"]

0     -0.350004
1      0.306186
2     -0.317425
3     -0.266172
4      0.306976
         ...   
635   -0.319956
636   -0.293362
637   -0.189388
638   -0.345815
639   -0.098954
Name: normalized_operation_cost, Length: 640, dtype: float64

SUM( 1️⃣ operation_cost , (2️⃣ poverty_rate*-1)

In [32]:
poverty["inverted_normalized_poverty_rate"] = poverty["normalized_poverty_rate"]*-1
poverty["inverted_normalized_poverty_rate"]

0        1.019323
1        1.019323
2        1.019323
3        1.019323
4        1.019323
           ...   
34789   -4.686590
34790   -4.686590
34791   -4.570616
34792   -4.570616
34793   -4.570616
Name: inverted_normalized_poverty_rate, Length: 34790, dtype: float64

Merge to be able to sum()

In [36]:
health_score = pd.merge(enviromental_health,poverty,on="municipality_code",how="left")
health_score

,municipality_code,city_name_x,operation_cost,normalized_operation_cost,poverty_rate,city_name_y,city_name_normalized,latitude,longitude,normalized_poverty_rate,inverted_normalized_poverty_rate
0,01004,Ambérieu-en-Bugey,251529.61,-0.350004,9.6,Ambérieu-en-Bugey,AMBERIEU EN BUGEY,45.960848,5.372926,-0.926544,0.926544
1,02142,Castres,5878227.00,0.306186,22.5,Castres,CASTRES,49.801389,3.253252,2.065581,-2.065581
2,02168,Château-Thierry,530884.00,-0.317425,15.1,Château-Thierry,CHATEAU THIERRY,49.056407,3.381600,0.349168,-0.349168
3,02372,Hartennes-et-Taux,970368.95,-0.266172,10.8,Hartennes-et-Taux,HARTENNES ET TAUX,49.278953,3.357633,-0.648207,0.648207
4,02422,Lesquielles-Saint-Germain,5885000.00,0.306976,23.8,Lesquielles-Saint-Germain,LESQUIELLES SAINT GERMAIN,49.931687,3.628086,2.367113,-2.367113
...,...,...,...,...,...,...,...,...,...,...,...
635,97414,Saint-Louis,509179.00,-0.319956,38.2,Saint-Louis,SAINT LOUIS,-21.233927,55.421339,5.707160,-5.707160
636,97415,Saint-Paul,737219.00,-0.293362,33.8,Saint-Paul,SAINT PAUL,-21.044532,55.322333,4.686590,-4.686590
637,97416,Saint-Pierre,1628779.45,-0.189388,38.2,Saint-Pierre,SAINT PIERRE,-21.312324,55.493616,5.707160,-5.707160
638,97418,Sainte-Marie,287442.47,-0.345815,33.3,Sainte-Marie,SAINTE MARIE,-20.947059,55.530356,4.570616,-4.570616


In [37]:
health_score.isnull().sum()

municipality_code                    0
city_name_x                          0
operation_cost                       0
normalized_operation_cost            0
poverty_rate                        17
city_name_y                         17
city_name_normalized                17
latitude                            17
longitude                           17
normalized_poverty_rate             17
inverted_normalized_poverty_rate    17
dtype: int64

Null values means that we don’t have the poverty rate of that 17 rows --> the best solution is to delete it

In [40]:
health_score.dropna(inplace=True)

we have city_name column duplicated it better to drop one

In [44]:
health_score.drop(columns="city_name_y",inplace=True)

In [49]:
health_score.duplicated().sum()

0

Finally the main measure is calculated!!

In [54]:
health_score["health_score"] = health_score["normalized_operation_cost"] + health_score["inverted_normalized_poverty_rate"]

In [55]:
health_score

,municipality_code,city_name_x,operation_cost,normalized_operation_cost,poverty_rate,city_name_normalized,latitude,longitude,normalized_poverty_rate,inverted_normalized_poverty_rate,health_score
0,01004,Ambérieu-en-Bugey,251529.61,-0.350004,9.6,AMBERIEU EN BUGEY,45.960848,5.372926,-0.926544,0.926544,0.576540
1,02142,Castres,5878227.00,0.306186,22.5,CASTRES,49.801389,3.253252,2.065581,-2.065581,-1.759395
2,02168,Château-Thierry,530884.00,-0.317425,15.1,CHATEAU THIERRY,49.056407,3.381600,0.349168,-0.349168,-0.666593
3,02372,Hartennes-et-Taux,970368.95,-0.266172,10.8,HARTENNES ET TAUX,49.278953,3.357633,-0.648207,0.648207,0.382035
4,02422,Lesquielles-Saint-Germain,5885000.00,0.306976,23.8,LESQUIELLES SAINT GERMAIN,49.931687,3.628086,2.367113,-2.367113,-2.060137
...,...,...,...,...,...,...,...,...,...,...,...
634,97411,Saint-Denis,752268.00,-0.291607,33.3,SAINT DENIS,-20.932971,55.446867,4.570616,-4.570616,-4.862223
635,97414,Saint-Louis,509179.00,-0.319956,38.2,SAINT LOUIS,-21.233927,55.421339,5.707160,-5.707160,-6.027116
636,97415,Saint-Paul,737219.00,-0.293362,33.8,SAINT PAUL,-21.044532,55.322333,4.686590,-4.686590,-4.979952
637,97416,Saint-Pierre,1628779.45,-0.189388,38.2,SAINT PIERRE,-21.312324,55.493616,5.707160,-5.707160,-5.896547


rename  city_name_x to city_name

In [58]:
health_score.rename(columns={
    "city_name_x" : "city_name"
},inplace=True)

In [59]:
health_score

,municipality_code,city_name,operation_cost,normalized_operation_cost,poverty_rate,city_name_normalized,latitude,longitude,normalized_poverty_rate,inverted_normalized_poverty_rate,health_score
0,01004,Ambérieu-en-Bugey,251529.61,-0.350004,9.6,AMBERIEU EN BUGEY,45.960848,5.372926,-0.926544,0.926544,0.576540
1,02142,Castres,5878227.00,0.306186,22.5,CASTRES,49.801389,3.253252,2.065581,-2.065581,-1.759395
2,02168,Château-Thierry,530884.00,-0.317425,15.1,CHATEAU THIERRY,49.056407,3.381600,0.349168,-0.349168,-0.666593
3,02372,Hartennes-et-Taux,970368.95,-0.266172,10.8,HARTENNES ET TAUX,49.278953,3.357633,-0.648207,0.648207,0.382035
4,02422,Lesquielles-Saint-Germain,5885000.00,0.306976,23.8,LESQUIELLES SAINT GERMAIN,49.931687,3.628086,2.367113,-2.367113,-2.060137
...,...,...,...,...,...,...,...,...,...,...,...
634,97411,Saint-Denis,752268.00,-0.291607,33.3,SAINT DENIS,-20.932971,55.446867,4.570616,-4.570616,-4.862223
635,97414,Saint-Louis,509179.00,-0.319956,38.2,SAINT LOUIS,-21.233927,55.421339,5.707160,-5.707160,-6.027116
636,97415,Saint-Paul,737219.00,-0.293362,33.8,SAINT PAUL,-21.044532,55.322333,4.686590,-4.686590,-4.979952
637,97416,Saint-Pierre,1628779.45,-0.189388,38.2,SAINT PIERRE,-21.312324,55.493616,5.707160,-5.707160,-5.896547


In [60]:
health_score.to_csv("health_score.csv",index=False)